# Baselines

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import helper
import matplotlib
import matplotlib.pyplot as plt
import w2v_d2v
import dataset_helper
import preprocessing
import classifier_baseline

In [8]:
clfs = classifier_baseline.get_classifiers(iterations=5)

def allowed_dataset(dataset_name):
    return dataset_name == 'ling-spam'

#clfs = {clf_name: clf for clf_name, clf in clfs.items() if clf_name == 'Doc2VecClassifier'}
for dataset_name, (X, Y) in datasets.items():
    if not allowed_dataset(dataset_name): continue
    #if not dataset_name.startswith('web'): continue
    print('{} {:<14} #docs {}'.format('=' * 30, dataset_name, len(X)))
    data_train_X, data_test_X, data_train_Y, data_test_Y = dataset_helper.split_dataset(X, Y)
    vectors_trans_train, vectors_trans_test = classifier_baseline.vectorize_text(data_train_X, data_test_X)
    pred_results = classifier_baseline.get_predictions_for_classifiers(vectors_trans_train, vectors_trans_test, data_train_Y, data_test_Y, data_train_X, data_test_X, clfs = clfs)
    results = classifier_baseline.get_f1_score_for_clfs(pred_results, data_train_Y, data_test_Y)
    display(pd.DataFrame(results).T.test.to_frame().sort_values('test'))
    #print('{}>    Best for dataset: {} is classifier: {} with F1-score of {}'.format('=' * 10, dataset_name, best_clf, best_clf_score['test']))

============================== ling-spam      #docs 2893
Currently training classifier: PassiveAggressiveClassifier    (5/5)


/Users/davidgengenbach/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,test
MostFrequentLabel,0.454802
LogisticRegression,0.924161
Perceptron,0.975232
SGDClassifier,0.993704
PassiveAggressiveClassifier,1.000000


In [22]:
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from transformers.wl_graph_kernel_transformer import WLGraphKernelTransformer
from transformers.preprocessing_transformer import PreProcessingTransformer
import graph_helper
import dataset_helper
import wl
import os

for dataset_name in dataset_helper.get_all_available_dataset_names():
    if dataset_name != 'ling-spam': continue
        
    X, Y = dataset_helper.get_dataset(dataset_name, use_cached= True)
    
    p = Pipeline([
        ('preprocessing', None),
        ('count_vectorizer', sklearn.feature_extraction.text.CountVectorizer()),
        ('TfidfTransformer', sklearn.feature_extraction.text.TfidfTransformer()),
        ('clf', sklearn.linear_model.PassiveAggressiveClassifier())
    ])
    
    param_grid = dict(
        preprocessing = [None, PreProcessingTransformer(only_nouns = True)],
        count_vectorizer__stop_words = ['english'],
        clf__n_iter = [100],
        clf__class_weight = ['balanced']
    )

    cv = sklearn.model_selection.StratifiedKFold(n_splits = 3, random_state= 42, shuffle= True)
    gscv = GridSearchCV(estimator = p, param_grid=param_grid, cv=cv, scoring = 'f1_macro', n_jobs=1, verbose = 11)
    gscv_result = gscv.fit(X, Y)
    print(gscv_result.best_estimator_, gscv_result.cv_results_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__class_weight=balanced, clf__n_iter=100, preprocessing=None, count_vectorizer__stop_words=english 
[CV]  clf__class_weight=balanced, clf__n_iter=100, preprocessing=None, count_vectorizer__stop_words=english, score=0.994423, total=   2.0s
[CV] clf__class_weight=balanced, clf__n_iter=100, preprocessing=None, count_vectorizer__stop_words=english 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s


[CV]  clf__class_weight=balanced, clf__n_iter=100, preprocessing=None, count_vectorizer__stop_words=english, score=0.986651, total=   2.0s
[CV] clf__class_weight=balanced, clf__n_iter=100, preprocessing=None, count_vectorizer__stop_words=english 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.1s remaining:    0.0s


[CV]  clf__class_weight=balanced, clf__n_iter=100, preprocessing=None, count_vectorizer__stop_words=english, score=0.992543, total=   1.9s
[CV] clf__class_weight=balanced, clf__n_iter=100, preprocessing=PreProcessingTransformer(only_nouns=True, return_text=True), count_vectorizer__stop_words=english 


...

[CV]  clf__class_weight=balanced, clf__n_iter=100, preprocessing=PreProcessingTransformer(only_nouns=True, return_text=True), count_vectorizer__stop_words=english, score=0.975080, total=  19.4s
[CV] clf__class_weight=balanced, clf__n_iter=100, preprocessing=PreProcessingTransformer(only_nouns=True, return_text=True), count_vectorizer__stop_words=english 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__class_weight=balanced, clf__n_iter=100, preprocessing=PreProcessingTransformer(only_nouns=True, return_text=True), count_vectorizer__stop_words=english, score=0.987047, total=  20.8s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.8min finished


Pipeline(steps=[('preprocessing', None), ('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor... n_iter=100, n_jobs=1,
              random_state=None, shuffle=True, verbose=0, warm_start=False))]) {'split0_train_score': array([ 1.,  1.]), 'param_count_vectorizer__stop_words': masked_array(data = ['english' 'english'],
             mask = [False False],
       fill_value = ?)
, 'split1_test_score': array([ 0.98665071,  0.97507989]), 'split1_train_score': array([ 1.,  1.]), 'split2_test_score': array([ 0.99254349,  0.98704736]), 'param_preprocessing': masked_array(data = [None PreProcessingTransformer(only_nouns=True, return_text=True)],
             mask = [False False],
       fill_value = ?)
, 'mean_score_time': array([ 0.55083807,  6.86513535]), 'rank_test_score': array(

In [25]:
import pandas as pd
df = pd.DataFrame(gscv_result.cv_results_)
df

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_clf__class_weight,param_clf__n_iter,param_count_vectorizer__stop_words,param_preprocessing,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,1.387270,0.550838,0.991207,1.0,balanced,100,english,None,"{'clf__class_weight': 'balanced', 'clf__n_iter...",1,0.994423,1.0,0.986651,1.0,0.992543,1.0,0.009371,0.013323,0.003311,0.0
1,13.576318,6.865135,0.980559,1.0,balanced,100,english,"PreProcessingTransformer(only_nouns=True, retu...","{'clf__class_weight': 'balanced', 'clf__n_iter...",2,0.979550,1.0,0.975080,1.0,0.987047,1.0,0.688925,0.056466,0.004937,0.0


## Train Doc2Vec and Word2Vec classifier

### Pre-process docs for d2v and w2v

In [59]:
w2v_data_train = [w2v_d2v.w2v_preproess(doc) for doc in data_train_X]
w2v_data_test = [w2v_d2v.w2v_preproess(doc) for doc in data_test_X]

### Train Doc2Vec

In [127]:
model_d2v = w2v_d2v.train_d2v(w2v_data_train, data_train_Y, iterations = 5)

In [128]:
scores = w2v_d2v.score_d2v(clfs, data_train_Y, data_test_Y, model_d2v, w2v_data_train, w2v_data_test, steps = 1)

Inferred train vectors
Inferred test vectors


/Users/davidgengenbach/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [129]:
scores

{'LogisticRegression': {'test': 0.43375480433862312,
  'train': 0.46532463660461809},
 'MostFrequentLabel': {'test': 0.0046355498721227621,
  'train': 0.0045970578829549087},
 'PassiveAggressiveClassifier': {'test': 0.26940955676406886,
  'train': 0.30912823359685837},
 'Perceptron': {'test': 0.006102700076132176, 'train': 0.010673521739491008},
 'SGDClassifier': {'test': 0.35247761491407559, 'train': 0.38979592547038749}}